In [1]:
import pymssql as mssql
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [2]:
def send_query(query):
    
    '''
    Function sending query to ODS
    Input: Query(String)
    Output: Dataframe wanted 
    '''    

    
    ods = mssql.connect(host = '128.110.13.89', 
                      user = '011553', 
                      password = 'Sino821031pac')

    odscur = ods.cursor(as_dict = True)
    odscur.execute(query)
    try:
        temp = odscur.fetchall()
    except:
        temp = []
        
    row_count = int(odscur.rowcount)
    df = pd.DataFrame(temp)
    ods.commit()
    ods.close()

    return df, row_count

In [3]:
st_query = '''WITH STAT_DAY AS(
SELECT RTRIM(LTRIM(S_IDNO)) AS S_IDNO,
        CASE WHEN ISDATE(DATE_YMD) = 1 THEN CONVERT(DATE, DATE_YMD) ELSE NULL END AS DATE_YMD,
		SUM(AMT_B) AS AMT_B,
        SUM(AMT_S) AS AMT_S
FROM ODS.dbo.STS_MATCH_BS_CUSTOMER_D
WHERE ISDATE(DATE_YMD) = 1 
		AND LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
		AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
		AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1
		AND RTRIM(LTRIM(S_IDNO)) IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) FROM ODS.dbo.ST_CUSTOMER)
		AND LEFT(DATE_YMD, 4) >= '2013'
		AND AMT_B + AMT_S != 0
GROUP BY RTRIM(LTRIM(S_IDNO)), DATE_YMD)
, CUSTOMER_INFO AS(
SELECT S_IDNO,
        RADDR_O,
        CADDR_O,
        GENDER,
        AGE
FROM
(SELECT RTRIM(LTRIM(S_IDNO)) AS S_IDNO,
        RTRIM(LTRIM(RADDR_O)) AS RADDR_O,
        RTRIM(LTRIM(CADDR_O)) AS CADDR_O,
		CASE WHEN S_IDNO LIKE '[A-Z]%' AND SUBSTRING(S_IDNO, 2, 1) = '1' THEN 'M'
			WHEN S_IDNO LIKE '[A-Z]%' AND SUBSTRING(S_IDNO, 2, 1) = '2' THEN 'F' ELSE 'N' END AS GENDER,
		CASE WHEN ISDATE(S_BIRTH) = 1 THEN DATEDIFF(YEAR, S_BIRTH, CONVERT(DATE, GETDATE())) ELSE NULL END AS [AGE],
        ROW_NUMBER () OVER (PARTITION BY RTRIM(LTRIM(S_IDNO)) ORDER BY S_ODATE DESC) AS [RANK]
FROM ODS.dbo.ST_CUSTOMER
WHERE LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
		AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
		AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1 
		AND ISDATE(S_BIRTH) = 1
		AND LTRIM(RTRIM(S_IDNO)) NOT IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) AS S_IDNO
											FROM ODS.dbo.FU_CUSTOMER
											WHERE LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
													AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
													AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1)
		AND LTRIM(RTRIM(S_IDNO)) NOT IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) AS S_IDNO
											FROM ODS.dbo.SUB_CUSTOMER
											WHERE LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
													AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
													AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1))D
WHERE [RANK] = 1)

SELECT MTH_DATE,
		S_IDNO,
		GENDER,
		AGE,
		AMT_B,
        AMT_S,
        BIRTH_ADDRESS,
        LIVING_ADDRESS,
		CASE WHEN DATEDIFF(DAY, MTH_DATE, NEXT_TRADES) < 360 THEN 'ACTIVE' ELSE 'CHURN' END AS LABEL_CHURN
FROM(
SELECT STAT_DAY.DATE_YMD AS MTH_DATE,
        CUSTOMER_INFO.S_IDNO AS S_IDNO,
		CUSTOMER_INFO.GENDER AS GENDER,
		CUSTOMER_INFO.AGE AS AGE,
        CUSTOMER_INFO.RADDR_O AS BIRTH_ADDRESS,
        CUSTOMER_INFO.CADDR_O AS LIVING_ADDRESS,
		FIRST_VALUE(STAT_DAY.DATE_YMD) OVER (PARTITION BY STAT_DAY.S_IDNO ORDER BY STAT_DAY.DATE_YMD ASC ROWS BETWEEN 1 FOLLOWING AND UNBOUNDED FOLLOWING) AS NEXT_TRADES,
		STAT_DAY.AMT_B AS AMT_B,
        STAT_DAY.AMT_S AS AMT_S
FROM STAT_DAY
INNER JOIN CUSTOMER_INFO ON STAT_DAY.S_IDNO = CUSTOMER_INFO.S_IDNO) A
WHERE MTH_DATE < DATEADD(DAY, -1, CONVERT(DATE, GETDATE()))
'''

In [4]:
st_df, rows = send_query(st_query)

In [5]:
st_df['MTH_DATE'] = pd.to_datetime(st_df['MTH_DATE'])
st_df['BIRTH_ADDRESS'] = st_df['BIRTH_ADDRESS'].apply(lambda x: x.replace('臺', '台'))
st_df['LIVING_ADDRESS'] = st_df['LIVING_ADDRESS'].apply(lambda x: x.replace('臺', '台'))
df = st_df[st_df['MTH_DATE'] <= st_df['MTH_DATE'].max() - timedelta(days=360)]
df['TRADE'] = 1
df

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,MTH_DATE,S_IDNO,GENDER,AGE,AMT_B,AMT_S,BIRTH_ADDRESS,LIVING_ADDRESS,LABEL_CHURN,TRADE
87,2019-01-28,A100000136,M,67,0,100250,台北市松山區自強里５鄰三民路６８巷４號５樓之１,台北市松山區三民路６８巷４號５樓之１,ACTIVE,1
88,2019-01-25,A100000136,M,67,0,35850,台北市松山區自強里５鄰三民路６８巷４號５樓之１,台北市松山區三民路６８巷４號５樓之１,ACTIVE,1
89,2018-11-23,A100000136,M,67,31900,0,台北市松山區自強里５鄰三民路６８巷４號５樓之１,台北市松山區三民路６８巷４號５樓之１,ACTIVE,1
90,2018-11-20,A100000136,M,67,164100,0,台北市松山區自強里５鄰三民路６８巷４號５樓之１,台北市松山區三民路６８巷４號５樓之１,ACTIVE,1
91,2018-11-15,A100000136,M,67,94500,95500,台北市松山區自強里５鄰三民路６８巷４號５樓之１,台北市松山區三民路６８巷４號５樓之１,ACTIVE,1
...,...,...,...,...,...,...,...,...,...,...
20603383,2018-02-26,Z200145633,F,17,121800,94600,連江縣北竿鄉塘岐村３鄰８２號,連江縣南竿鄉清水村１２８之３號４樓,ACTIVE,1
20603384,2017-04-10,Z200145633,F,17,35300,0,連江縣北竿鄉塘岐村３鄰８２號,連江縣南竿鄉清水村１２８之３號４樓,ACTIVE,1
20603385,2016-11-29,Z200145633,F,17,33700,0,連江縣北竿鄉塘岐村３鄰８２號,連江縣南竿鄉清水村１２８之３號４樓,ACTIVE,1
20603386,2016-10-14,Z200145633,F,17,34600,0,連江縣北竿鄉塘岐村３鄰８２號,連江縣南竿鄉清水村１２８之３號４樓,ACTIVE,1


In [6]:
df.to_csv(r'D:\Customer_Value\data\st_trading.csv', index=False)